In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras_tuner

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

data = data.drop(
    columns=["release_date", "key", "loudness",
             "explicit", ])

data.head(5)

In [ ]:
# Combine genres and favourite_genres
all_genres = list(data['favourite_genres'] + data['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
encoded_all_genres = mlb.fit_transform(all_genres)

# Split encoded_genres into genres and favourite_genres
encoded_favourite_genres = encoded_all_genres[:, :len(data['favourite_genres'][0])]
encoded_genres = encoded_all_genres[:, len(data['favourite_genres'][0]):]

In [ ]:
# Concatenate the one-hot encoded columns
X = np.concatenate([encoded_favourite_genres, encoded_genres], axis=1)
# Extract the labels
y = data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(6000, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(3000, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1000, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(
      optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
epochs = 25
batch_size = 32

# history = model.fit(
#     X_train,
#     y_train,
#     validation_split=0.2,
#     epochs=epochs,
#     batch_size=batch_size,
#     verbose=2
# )

tuner = keras_tuner.tuners.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir')


In [ ]:
tuner.search(
    (X_train, y_train),
    validation_data=(X_test, y_test),
)

In [ ]:
# # Evaluate the model on the test set
# loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
# print("Test set accuracy:", accuracy)
#
# print("TEST")
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
# print("Accuracy:", accuracy_score(y_test, y_pred_classes))
# print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
# print("Classification report:\n", classification_report(y_test, y_pred_classes))
#
# print("TRAIN")
# y_pred = model.predict(X_train)
# y_pred_classes = (y_pred > 0.5).astype(int)
# print("Accuracy:", accuracy_score(y_train, y_pred_classes))
# print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
# print("Classification report:\n", classification_report(y_train, y_pred_classes))

In [ ]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [ ]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
